O arquivo ex5-files.zip gera um diretório de diretórios (com as classes) e 
5000 textos divididos nos subdiretórios apropriados. Esse é o formato para 
a função sklearn.datasets.load_files

O arquivo category.tab contém a classe de cada documento.

Os textos sao parte de um data set de mineração de documentos com textos de 
tamanho médio sobre tecnlogia. As classes sao as classes originais dos textos.

<b>Parte 1 - processamento de texto</b>

Faça as tarefas usuais de processamento de textos:

- conversão de caracteres maiúsculos para minúsculos
- remoção de pontuação
- remoção de stop words
- steming dos termos
- remoção dos termos que aparecem em um só documento ou que aparecem em todos.

Converta os textos processados acima em um bag of words no formato binário (0/1 
se o termo aparece ou não aparece no documento) e no formato de term frequency.

Em Python, o sklearn tem funçẽes para gerar as diferentes formas da matriz 
termo-documento. 

O preprocessamento acima deve ser feito para todos os textos em conjunto. Não é 
preciso fazer a separação de fit no conjunto de treino e transform no conjunto 
de teste.

Divida o conjunto em 1000 documentos de teste e 4000 de treino aleatoriamente 
(pode ser estratificado ou nao).


In [ ]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

# Carrega os dados 
dataset = load_files(
        container_path= "/home/mauricio/projects/MO444/lista 5/filesk", 
        encoding='utf-8', random_state=1)
# Conjunto de dados
X = dataset.data
# Label
y = dataset.target

As classes CountVectorizer e TfidfVectorizer podem realizar todos os 
preprocessamentos pedidos no exercício, menos o steming dos termos. Portanto foi 
criado uma classe Tokenizer, para substituir o processo de tokenização padrão e 
usar a parte de steming no nltk <b>SnowballStemmer</b>.
A Classe CountVectorizer criará um Bag of words binário e a TfidfVectorizer 
criará um Bag of words com term-frequency.

Por default as classes CountVectorizer e TfidfVectorizer já fazem:
- remoção de pontuação (e considerar apenas tokens de pelo menos 2 letras
- conversão de caracteres maiúsculos para minúsculos 
(parâmetro <b>lowercase</b> pr default = <b>True</b>)

E configuramos os que não são default:
- remoção de stop words <b>stop_words = 'english</b>
- steming dos termos <b>tokenizer=Tokenizer()</b>
- remoção dos termos que aparecem em um só documento ou que aparecem em todos.
<b>min_df = 2 e  max_df=0.99999</b> . 

O problemas desses parâmetros é que o valor
não da condição não é um intervalo fechado, então para remover os termos que só 
aparecem 1 vez temos que configurar o <b>min_df</b> para q todos os valores 
menores que <b>min_df</b> sejam removidos ( e não menor ou igual).

Para remover os termos que aparecem em todos os documentos temos que configurar 
o <b>max_df</b> para que todos os valores maiores que <b>max_df</b> sejam 
removidos (e não maior ou igual)

A Classe CounterVectorizer foi configurada também para binarizar e não contar as 
ocorrências das palavras (<b>binary=True</b>)                              


In [ ]:
# Essa classe substituirá a parte de tokenização padrão do CountVectorizer e 
# TfidfVectorizer, porque precisamos usar o stem do nltk
class Tokenizer(object):
    def __init__(self):
        self.tokenizer = CountVectorizer()
        self.stemmer = SnowballStemmer("english")
    def __call__(self, doc):
        tokens = self.tokenizer.build_tokenizer()(doc)
        return [self.stemmer.stem(token) for token in tokens]

# Bag of words no formato binário
binaryVectorizer = CountVectorizer(tokenizer=Tokenizer(),
                                    min_df=2, 
                                    max_df=0.99999, 
                                    binary=True,
                                    stop_words = 'english', 
                                    strip_accents = 'unicode')

# Bag of words no formato de term frequency.
tfidVectorizer = TfidfVectorizer(tokenizer=Tokenizer(), 
                                 min_df=2, 
                                 max_df=0.99999, 
                                 stop_words = 'english', 
                                 strip_accents = 'unicode')


Crio os conjuntos de treino e testes para o binário e para o Term-frequency

In [ ]:
# Binariza 
X_binary = binaryVectorizer.fit_transform(X)
# Cria conjunto de teste e treino
split_binary = train_test_split(X_binary, y, train_size=4000, 
                                test_size=1000, random_state=1)
X_train_binary,X_test_binary,y_train_binary,y_test_binary = split_binary

# Transforma em Term-frequency
X_tfi = tfidVectorizer.fit_transform(X)
# Cria conjunto de teste e treino
split_tfi = train_test_split(X_tfi, y, train_size=4000, 
                                test_size=1000, random_state=1)
X_train_tfi, X_test_tfi, y_train_tfi, y_test_tfi = split_tfi

<b>Parte 2 - naive bayes</b>

Rode o naive bayes (BernoulliNB) na matriz binária. Qual a acurácia?

Rode o naive Bayes (MultinomialNB) na matriz de term-frequency. Qual a acurácia
(compare com a anterior).

In [ ]:
bBernoulli_classifier = BernoulliNB()
bBernoulli_classifier.fit(X_train_binary, y_train_binary)
print("score BernoulliNB = %s" % bBernoulli_classifier.score(
        X = X_test_binary, y = y_test_binary))

multinomialNB_classifier = MultinomialNB()
multinomialNB_classifier.fit(X_train_tfi, y_train_tfi)
print("score MultinomialNB = %s" % multinomialNB_classifier.score(
        X=X_test_tfi, y = y_test_tfi))

<b>score BernoulliNB = 0.756</b></p>
<b>score MultinomialNB = 0.602</b>

A acurácia do BernoulliNB com a matriz binária apresentou melhores resultados do 
que MultinomialNB com Term-frequency

<b>Parte 3 -PCA e outros classificadores</b>

Rode o PCA e reduza o número de dimensões da matriz de term-frequency para 99% 
da variância original. Você não consiguirá usar o PCA tradicional do Sklearn. 
Voce terá que usar o TuncatedSVD que é menos conveniente.

Rode SVM com RBF (modo one vs all), gradient boosting e random forest na matriz 
com o número de dimensões reduzidas. Não se esqueça de fazer a busca de 
hiperparâmetros. Quais as acurácias?

Qual o melhor classificador dos testados?

In [ ]:
pca = TruncatedSVD(n_components=3035, n_iter=10, random_state=1)
pca.fit(X_train_tfi)    
print(pca.explained_variance_ratio_.sum())

X_train_tfi_pca = pca.transform(X_train_tfi)
X_test_tfi_pca = pca.transform(X_test_tfi)

Para a variância de 99% do original foi usado o <b>n_components = 3035</b> e 
<b>n_iter = 10</b><p>

<b>0.990019468958</b>

Como estamos usando o OneVsRestClassifier e o SVC está sendo passado como 
parâmetro e na verdade queremos estimar os parâmetros do SVC temos que 
passar para o Grid Search os parâmetros a serem encontrados com o prefixo
<b>estimator__</b>

In [ ]:
# estimator__ como prefixo do parâmetro serve para entender que ele não é 
# do OneVsRestClassifier e sim do SVC
p_svm_grid = [{'estimator__C': [2**(-5), 2**(0), 2**(5), 2**(10)],
      'estimator__gamma': [2**(-15), 2**(-10), 2**(-5), 2**(0), 2**(5)]}]

grid_svm = GridSearchCV(
        estimator=OneVsRestClassifier(SVC(random_state=1)), 
        param_grid=p_svm_grid,
        n_jobs = 8)
grid_svm.fit(X_train_tfi_pca, y_train_tfi)

ovr_svm = OneVsRestClassifier(
        SVC(C = grid_svm.best_params_.get('estimator__C'),
            gamma = grid_svm.best_params_.get('estimator__gamma')),
        n_jobs = 8)
ovr_svm.fit(X_train_tfi_pca, y_train_tfi)

print("score tf-idf SVM = %s" % ovr_svm.score(
        X = X_test_tfi_pca, y= y_test_tfi))

<b>Best C = 1</b>

<b>Best Gamma = 1</b>

<b>score tf-idf SVM = 0.854</b>

In [ ]:
p_gb_grid = [{'n_estimators': [30, 70, 100],
           'learning_rate': [0.1, 0.05]}]
    
grid_gb = GridSearchCV(
       estimator=GradientBoostingClassifier(random_state=1, max_depth = 5), 
       param_grid=p_gb_grid,
       n_jobs = 8)
grid_gb.fit(X_train_tfi_pca, y_train_tfi)

gb = GradientBoostingClassifier(
        n_estimators = grid_gb.best_params_.get('n_estimators'),
        learning_rate = grid_gb.best_params_.get('learning_rate'),
        max_depth = 5)
gb.fit(X_train_tfi_pca, y_train_tfi)

print("score tf-idf GradientBoosting = %s" % gb.score(
        X = X_test_tfi_pca, y = y_test_tfi))

<b>Best n_estimators = 100</b>

<b>Best learning_rate = 0.1</b>

<b>sscore tf-idf GradientBoosting = 0.839</b>

In [ ]:
param_grid_rf = [{'n_estimators': [100, 200, 400, 800], 
                  'max_features': [1000, 1500, 2000, 3000]}]
grid_rf = GridSearchCV(
        estimator=RandomForestClassifier(random_state=1), 
        param_grid = param_grid_rf, 
        n_jobs = 8)
grid_rf.fit(X_train_tfi_pca, y_train_tfi)

rf = RandomForestClassifier(
        n_estimators = grid_rf.best_params_.get('n_estimators'),
        max_features = grid_rf.best_params_.get('max_features'),
        random_state =  1,
        n_jobs = 8)
rf.fit(X_train_tfi_pca, y_train_tfi)

print("score tf-idf Random Forest = %s" % rf.score(
        X = X_test_tfi_pca, y = y_test_tfi))

<b>Best n_estimators = 200</b>

<b>Best max_features = 1500</b>

<b>score tf-idf Random Forest = 0.823</b>

Como podemos ver nos resultados obtidos o SVM usando a técnica de One vs Rest 
obteve os melhores resultados. A busca por parâmetros e o treinamento para
OneVsRestClassifier, GradientBoostingClassifier e RandomForestClassifier 
demoraram horas, mesmo em alguns casos configurando o algoritmo para usar
8 cores de processamento (n_jobs = 8)